# Download, extract and store data locally

# User params

In [2]:
# Possible categories to download:
# --------------------------------
# All_Beauty
# Amazon_Fashion
# Appliances
# Arts_Crafts_and_Sewing
# Automotive
# Baby_Products
# Beauty_and_Personal_Care
# Books
# CDs_and_Vinyl
# Cell_Phones_and_Accessories
# Clothing_Shoes_and_Jewelry
# Digital_Music
# Electronics
# Gift_Cards
# Grocery_and_Gourmet_Food
# Handmade_Products
# Health_and_Household
# Health_and_Personal_Care
# Home_and_Kitchen
# Industrial_and_Scientific
# Kindle_Store
# Magazine_Subscriptions
# Movies_and_TV
# Musical_Instruments
# Office_Products
# Patio_Lawn_and_Garden
# Pet_Supplies
# Software
# Sports_and_Outdoors
# Subscription_Boxes
# Tools_and_Home_Improvement
# Toys_and_Games
# Video_Games

category_to_download = "Gift_Cards"


# For dataset library
# ---------------------
# type of data to download
# entire unprocessed data: 
# - 'raw_'
# processed data : 
# - "0core_rating_", "0core_timestamp_", "0core_timestamp_w_his_"
# - "5core_rating_", "5core_timestamp_", "5core_timestamp_w_his_"
type = 'raw_'
split = "full" # "full", "train", "valid", "test"


# Utility Functions

In [3]:
import os, sys
sys.path.append(os.path.abspath('../src'))

# other imports
from pathlib import Path

# Processing - native

In [3]:
# import os
# from src.config import BASE_PATH_DATA
# from src.utils.io.io import load_dataframe

# from src.utils.io.download import download_data
# download_data(category_to_download)

# # load data
# file_name_review = category_to_download + "_review.json"
# file_name_metadata = category_to_download + "_metadata.json"

# # read json file into dataframe
# df_metadata = load_dataframe(os.path.join(BASE_PATH_DATA, 'raw', file_name_review))
# df_review = load_dataframe(os.path.join(BASE_PATH_DATA, 'raw', file_name_metadata))


# # Merge the datasets on 'parent_asin' with suffixes for duplicate columns
# merged_df = pd.merge(df_review, df_metadata, 
#                      on="parent_asin", how="inner", suffixes=("_review", "_metadata"))

2024-08-05 15:10:54,363 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): datarepo.eng.ucsd.edu:443


-- download reviews --


2024-08-05 15:10:54,947 - urllib3.connectionpool - DEBUG - https://datarepo.eng.ucsd.edu:443 "GET /mcauley_group/data/amazon_2023/raw/review_categories/Gift_Cards.jsonl.gz HTTP/11" 200 12556849
2024-08-05 15:11:04,122 - src.config - INFO - Decompressed and saved to ..\data\raw\Gift_Cards_review.json
2024-08-05 15:11:04,205 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): datarepo.eng.ucsd.edu:443


Downloaded and converted Gift_Cards_review.json to ..\data\raw
-- download metadata --


2024-08-05 15:11:04,804 - urllib3.connectionpool - DEBUG - https://datarepo.eng.ucsd.edu:443 "GET /mcauley_group/data/amazon_2023/raw/meta_categories/meta_Gift_Cards.jsonl.gz HTTP/11" 200 324339
2024-08-05 15:11:05,704 - src.config - INFO - Decompressed and saved to ..\data\raw\Gift_Cards_metadata.json


Downloaded and converted Gift_Cards_metadata.json to ..\data\raw


# Processing - using datasets library

In [7]:
from datasets import load_dataset

dataset_reviews = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_" + category_to_download, trust_remote_code=True)
print(dataset_reviews[split].num_rows)
print(dataset_reviews[split][0])

dataset_medata = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_"+ category_to_download, split=split, trust_remote_code=True)
print(dataset_medata.num_rows)
print(dataset_medata[0])

152410
{'rating': 5.0, 'title': 'Great gift', 'text': 'Having Amazon money is always good.', 'images': [], 'asin': 'B00IX1I3G6', 'parent_asin': 'B00IX1I3G6', 'user_id': 'AHZ6XMOLEWA67S3TX7IWEXXGWSOA', 'timestamp': 1549866158332, 'helpful_vote': 0, 'verified_purchase': True}
1137
{'main_category': 'Gift Cards', 'title': 'Amazon.com Gift Card in Gift Tag (Various Designs)', 'average_rating': 4.8, 'rating_number': 1006, 'features': ['Gift Card is affixed inside a gift tag', 'Gift amount may not be printed on Gift Cards', 'Gift Card has no fees and no expiration date', 'No returns and no refunds on Gift Cards', 'Gift Card is redeemable towards millions of items storewide at Amazon.com', 'Scan and redeem any Gift Card with a mobile or tablet device via the Amazon App', 'Free One-Day Shipping (where available)', 'Customized gift message, if chosen at check-out, only appears on packing slip and not on the actual gift card or carrier'], 'description': ["Amazon.com Gift Cards are the perfect wa

In [14]:
import pandas as pd
import os

# Tranform to pandas
df_review = pd.DataFrame(dataset_reviews[split])
df_metadata = pd.DataFrame(dataset_medata)

# merge
df = pd.merge(df_review, df_metadata, on="parent_asin", how="inner", suffixes=("_review", "_metadata"))

# drop duplicates not to have data leakage
# from src.utils.preprocessing.duplicates import drop_duplicates_with_unhashable
# merged_df = drop_duplicates_with_unhashable(merged_df, ['user_id', 'timestamp'])
subset_columns = ['rating', 'title_review', 'text','asin',
       'parent_asin', 'user_id', 'timestamp', 'helpful_vote',
       'verified_purchase', 'main_category', 'title_metadata',
       'average_rating', 'rating_number', 'price',
        'store', 'bought_together', 'subtitle', 'author']
df.drop_duplicates(subset=subset_columns, inplace=True)

# Find the most frequent label in the main_category column and
# filter the DataFrame to keep only rows with the most frequent label (some were in wrong category)
main_label = df["main_category"].value_counts().idxmax()
df = df[df["main_category"] == main_label]

# Save it

In [15]:
dest_folder = Path("../data/raw")
save_name = os.path.join(dest_folder, f'merged_dataset.parquet')
df.to_parquet(save_name, index=False)